This notebook contains code to process salinity data that has been downloaded from:
- https://data.marine.copernicus.eu/product/GLOBAL_ANALYSISFORECAST_PHY_001_024/download?dataset=cmems_mod_glo_phy-so_anfc_0.083deg_P1D-m_202406. 

A two week sample of that data over a region of interest is avaiable in ./environmentalData for between 5 and 10 m. 

Tool 1: takes the .nc file and outputs geojson depth files of the whole timeframe. The geojson is readable by geoPandas, and can then be used to both visulize the variable in a given area and is available for interrogation by other layers and processes. Grid spacing is 0.083 degrees. 

Tool 2: takes the .nc file and outputs a gif of a given depth over the timeframe. This is just for fun!

In [2]:
"""
Tool 1: depth slices
"""

import xarray as xr
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
import os

# Replace with your actual NetCDF file path if different
home_dir = os.getcwd()
file_path = os.path.join(home_dir, 'environmentalData', 'cmems_mod_glo_phy-so_anfc_0.083deg_P1D-m_1749908476181.nc')

# Open the dataset
ds = xr.open_dataset(file_path)

# Extract salinity variable
salinity = ds['so']

# Get all depth values
depth_values = ds['depth'].values

# Create output directory
output_dir = os.path.join(home_dir, 'geojson_by_depth')
os.makedirs(output_dir, exist_ok=True)

# Loop through each depth level
for depth_val in depth_values:
    # Select salinity at this depth
    salinity_at_depth = salinity.sel(depth=depth_val)

    # Convert to DataFrame and reset index
    df = salinity_at_depth.to_dataframe().reset_index().dropna(subset=['so'])

    # Add depth column
    df['depth'] = float(depth_val)

    # Create geometry from lat/lon
    geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]

    # Create GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry=geometry)

    # Keep relevant columns
    gdf = gdf[['so', 'time', 'depth', 'geometry']]

    # Define output file path
    output_file = os.path.join(output_dir, f'salinity_depth_{depth_val:.3f}m.geojson')

    # Export to GeoJSON
    gdf.to_file(output_file, driver='GeoJSON')

    # Print confirmation
    print(f"Exported GeoJSON for depth {depth_val:.3f} m to {output_file}")


Exported GeoJSON for depth 5.078 m to C:/Users/tdow214/PycharmProjects/OceanData/geojson_by_depth\salinity_depth_5.078m.geojson
Exported GeoJSON for depth 6.441 m to C:/Users/tdow214/PycharmProjects/OceanData/geojson_by_depth\salinity_depth_6.441m.geojson
Exported GeoJSON for depth 7.930 m to C:/Users/tdow214/PycharmProjects/OceanData/geojson_by_depth\salinity_depth_7.930m.geojson
Exported GeoJSON for depth 9.573 m to C:/Users/tdow214/PycharmProjects/OceanData/geojson_by_depth\salinity_depth_9.573m.geojson


In [3]:
"""
Tool 2: GIF through time of the data for a given depth
"""

import xarray as xr
import matplotlib.pyplot as plt
import os
import pandas as pd
import imageio.v2 as imageio

# Set your NetCDF file path
home_dir = os.getcwd()
file_path = os.path.join(home_dir, 'environmentalData', 'cmems_mod_glo_phy-so_anfc_0.083deg_P1D-m_1749908476181.nc')

# Output directory for daily plots
output_dir = 'salinity_frames'
os.makedirs(output_dir, exist_ok=True)

# Open dataset
ds = xr.open_dataset(file_path)

# Choose a fixed depth index (e.g., surface)
depth_index = 0
depth_value = float(ds['depth'].isel(depth=depth_index).values)

# Store image paths
image_files = []

# Loop through each time step
for t in range(ds.sizes['time']):
    salinity = ds['so'].isel(time=t, depth=depth_index)
    time_value = pd.to_datetime(ds['time'].isel(time=t).values)

    df = salinity.to_dataframe().reset_index().dropna(subset=['so'])

    fig, ax = plt.subplots(figsize=(10, 8))
    sc = ax.scatter(df['longitude'], df['latitude'], c=df['so'], cmap='viridis', s=10)
    plt.colorbar(sc, ax=ax, label='Salinity (PSU)')
    ax.set_title(f'Salinity at {depth_value:.3f} m on {time_value.date()}')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    plt.grid(True)

    image_path = os.path.join(output_dir, f'salinity_{t:03d}.png')
    plt.savefig(image_path)
    plt.close()
    image_files.append(image_path)

# Create animated GIF
gif_path = 'salinity_animation.gif'
with imageio.get_writer(gif_path, mode='I', duration=0.8) as writer:
    for filename in image_files:
        image = imageio.imread(filename)
        writer.append_data(image)

print(f"Animated GIF saved as {gif_path}")

Animated GIF saved as salinity_animation.gif
